In [1]:
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serve import SchemaBuilder
from sagemaker.serve import ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode
import mlflow
from mlflow import MlflowClient
import boto3
import numpy as np
import pandas as pd
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
# Define session, role, and region so we can
# perform any SageMaker tasks we need
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name

# S3 prefix for the training dataset to be uploaded to
prefix = 'DEMO-scikit-iris'

# Provide the ARN of the Tracking Server that you want to track your training job with
tracking_server_arn = "arn:aws:sagemaker:eu-west-1:211125740051:mlflow-tracking-server/Sample-server"

In [4]:
!mkdir -p training_code

In [5]:
os.makedirs("./data", exist_ok=True)

s3_client = boto3.client("s3")
s3_client.download_file(
    f"sagemaker-example-files-prod-{region}", 'datasets/tabular/iris/iris.data', './data/iris.csv'
)

df_iris = pd.read_csv('./data/iris.csv', header=None)
df_iris[4] = df_iris[4].map({"Iris-setosa": 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
iris = df_iris[[4, 0, 1, 2, 3]].to_numpy()
np.savetxt('./data/iris.csv', iris, delimiter=',', fmt='%1.1f, %1.3f, %1.3f, %1.3f, %1.3f')

In [6]:
WORK_DIRECTORY = 'data'

train_input = sagemaker_session.upload_data(
    WORK_DIRECTORY, key_prefix='{}/{}'.format(prefix, WORK_DIRECTORY)
)

In [7]:
%%writefile training_code/train.py

from __future__ import print_function

import argparse
import joblib
import os
import pandas as pd

from sklearn import tree

import mlflow

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) if os.path.isfile(os.path.join(args.train, file))]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    train_data = pd.concat(raw_data)

    # Set the Tracking Server URI using the ARN of the Tracking Server you created
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_ARN'])
    
    # Enable autologging in MLflow
    mlflow.autolog()

    # labels are in the first column
    train_y = train_data.iloc[:, 0]
    train_X = train_data.iloc[:, 1:]

    # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
    # as your training my require in the ArgumentParser above.
    max_leaf_nodes = args.max_leaf_nodes

    # Now use scikit-learn's decision tree classifier to train the model.
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(train_X, train_y)

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))
    
    # Register the model with MLflow
    run_id = mlflow.last_active_run().info.run_id
    artifact_path = "model"
    model_uri = "runs:/{run_id}/{artifact_path}".format(run_id=run_id, artifact_path=artifact_path)
    model_details = mlflow.register_model(model_uri=model_uri, name="sm-job-experiment-model")

Writing training_code/train.py


In [8]:
%%writefile training_code/requirements.txt
mlflow==2.13.2
sagemaker-mlflow==0.1.0
cloudpickle==2.2.1 # Required for Sagemaker Python SDK

Writing training_code/requirements.txt


In [11]:
sklearn = SKLearn(
    entry_point='train.py',
    source_dir='training_code',
    framework_version='1.2-1',
    instance_type='ml.m5.large',
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={'max_leaf_nodes': 30},
    keep_alive_period_in_seconds=3600,
    environment={
        'MLFLOW_TRACKING_ARN': tracking_server_arn
    }
)

In [12]:
sklearn.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-06-26-11-08-47-145


2024-06-26 11:08:47 Starting - Starting the training job...
2024-06-26 11:09:01 Starting - Preparing the instances for training...
2024-06-26 11:09:28 Downloading - Downloading input data...
2024-06-26 11:09:53 Downloading - Downloading the training image.....2024-06-26 11:10:58,298 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-26 11:10:58,302 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-26 11:10:58,305 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-26 11:10:58,321 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-26 11:10:58,532 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt

2024-06-26 11:10:53 Training - Training image download completed. Training in progress.Collecting fonttools>=4.22.0 (from matplotlib<4->mlflow=

In [13]:
mlflow.set_tracking_uri(tracking_server_arn)
client = MlflowClient()
registered_model = client.get_registered_model(name="sm-job-experiment-model")
source_path = registered_model.latest_versions[0].source

In [14]:
# Artifact URI of the model
source_path

's3://sagemaker-bucket-ds/ML_FLOW/0/5f5d29fc7fb8472094ee148ce7127d26/artifacts/model'

In [15]:
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0]).reshape(1, -1)
sklearn_output = 1
sklearn_schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)

In [16]:
# Create model builder with the schema builder.
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=sklearn_schema_builder,
    role_arn=role,
    model_metadata={
        "MLFLOW_MODEL_PATH": source_path
    }
)

In [17]:
built_model = model_builder.build()

ModelBuilder: WARNING:     Support of MLflow format models is experimental and is not intended for production at this moment.


ModelBuilder: INFO:     Auto-detected deployment flavor is sklearn
ModelBuilder: INFO:     Auto-detected framework version is 1.2.1
ModelBuilder: INFO:     Auto detected 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3. Proceeding with the the deployment.
ModelBuilder: INFO:     Skipping auto detection as the image uri is provided 141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
ModelBuilder: DEBUG:     Packaging provided requirements.txt from /tmp/sagemaker/model-builder/a3edc70e33ab11efa8fce600116ddf77/requirements.txt


ModelBuilder: DEBUG:     Uploading the model resources to bucket=sagemaker-eu-west-1-211125740051, key_prefix=sagemaker-scikit-learn-2024-06-26-11-15-27-566.
Uploading model artifacts: 100%|████████████████████████| 11950/11950 [00:00<00:00, 18148.59bytes/s]
ModelBuilder: DEBUG:     Model resources uploaded to: s3://sagemaker-eu-west-1-211125740051/sagemaker-scikit-learn-2024-06-26-11-15-27-566/serve.tar.gz


In [18]:
predictor = built_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-06-26-11-16-19-747
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-06-26-11-16-20-351
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-06-26-11-16-20-351


-----------!

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


In [19]:
predictor.predict(sklearn_input)

[2.0]

In [21]:
sagemaker_session.delete_endpoint(endpoint_name=built_model.endpoint_name)

INFO:sagemaker:Deleting endpoint with name: sagemaker-scikit-learn-2024-06-26-11-16-20-351
